In [11]:
# imports
import pandas as pd
import random

import os

In [12]:
# read in decklist
# WE ARE RUNNING 21 SWAMPS

decklist = pd.read_csv(os.path.join('data', 'gonti,_night_minister.csv'))

sub_ndxs = [0,1,2,3]

decklist.iloc[sub_ndxs,:]

,Unnamed: 0,name,mana_cost,cmc,colors,type_line,oracle_text,produced_mana
0,77,"Shizo, Death's Storehouse",NaN,0.0,[],Legendary Land,"{T}: Add {B}.\n{B}, {T}: Target legendary crea...",['B']
1,497,Diabolic Intent,{1}{B},2.0,['B'],Sorcery,"As an additional cost to cast this spell, sacr...",NaN
2,533,Misinformation,{B},1.0,['B'],Instant,Put up to three target cards from an opponent'...,NaN
3,610,Deadly Rollick,{3}{B},4.0,['B'],Instant,"If you control a commander, you may cast this ...",NaN


In [13]:
# reading in text files
# as a heads up you need to delete the last line of this as its 

def read_decklist_txt(path) -> pd.DataFrame:
    with open(path, 'r') as f:
        string = f.read()
        string = string.split('\n')
        # first index is number
        numbers = [i.split(' ')[0] for i in string]

        # read categories

        cats = [i[i.find('['):i.find(']')+1].strip('[]')
                for i in string]
        cats = [i.split(',') for i in cats]

        # even out number of categories
        max_cat_len = max([len(c) for c in cats])
        cats = [i + (['NA'] * (max_cat_len-len(i)))
                for i in cats]

        # read color tags

        tags = [i[i.find('^'):-1].strip('^') for i in string]
        tags = [i.split(',')[0] for i in tags] # drop colors

        # get names
        names = [i[i.find(' ')+1:i.find('[')-1] for i in string]

        columns = sum(
            [['name'], [f'cat_{ndx}' for ndx in range(max_cat_len)],
             ['tags'], ['no']], []
        )

        decklist = pd.DataFrame(
            [sum([[names[ndx]], cats[ndx], [tags[ndx]], [numbers[ndx]]], [])
             for ndx in range(len(string))],
             columns=columns
        )

        return decklist

In [26]:
def get_decklist_info(card_data:pd.DataFrame, decklist:pd.DataFrame):
    """Map card data to a decklist"""
    return card_data[card_data['name'].isin(decklist['name'])]

decklist = read_decklist_txt(os.path.join('data', 'gonti,_night minister.txt'))

card_path = os.path.join('data', 'oracle-cards-20250414210533.json')

card_data = pd.read_json(card_path)
card_data.index.names = ['scryfall_ndx']
card_data['decklist_ndx'] = [i for i in range(card_data.shape[0])]
card_data = card_data.set_index('decklist_ndx')

# decklist = pd.concat(
#     [decklist, get_decklist_info(pd.read_json(card_path), decklist)],
#     axis = 0
# )
get_decklist_info(card_data, decklist)

,object,id,oracle_id,multiverse_ids,mtgo_id,tcgplayer_id,cardmarket_id,name,lang,released_at,...,card_faces,produced_mana,tcgplayer_etched_id,loyalty,life_modifier,hand_modifier,attraction_lights,color_indicator,content_warning,defense
decklist_ndx,,,,,,,,,,,,,,,,,,,,,
77,card,099352e2-38c8-4fb4-a25f-6d928aa20f9e,008f2698-1721-45a3-8353-10f2f400dc8f,[578849],103356.0,283016.0,672215.0,"Shizo, Death's Storehouse",en,2022-09-09,...,NaN,[B],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
497,card,d72ab698-de67-4ca8-8e42-b05346bf52fa,038519b9-bca8-4b27-b5ac-2409595469d0,[583674],104796.0,452124.0,682332.0,Diabolic Intent,en,2022-11-18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
533,card,90a79cd5-d951-4ad7-a6a1-0cade4d0c189,03dc8820-ec10-42c1-8200-d323cfcbf2b0,[184698],30696.0,NaN,NaN,Misinformation,en,2008-09-22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
610,card,0e13f735-54fa-42b6-aea4-ced33811d7d4,0456ec64-2c81-4763-a352-8ff64a4c3d6b,[622755],115393.0,503565.0,721890.0,Deadly Rollick,en,2023-08-04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
665,card,38834333-f9ea-43a8-b00a-17935d59eaea,04b7362d-0490-4cb0-b5d7-2a7732f659ce,[696497],138623.0,624293.0,818911.0,Bojuka Bog,en,2025-04-11,...,NaN,[B],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32586,card,3573b9a2-7911-475c-8ae7-25bd0dbb7159,f1c2dbe2-fbe0-4058-bdf1-91d1b1832786,[696332],138289.0,624420.0,818785.0,Dauthi Voidwalker,en,2025-04-11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32752,card,e9e4ff9c-6244-43e1-b569-32a270c4e014,f2fcef65-4873-4b1f-a257-b8648097c990,[599098],107977.0,462150.0,688783.0,Helm of Awakening,en,2023-01-13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33290,card,32af7061-fca6-4a06-be3c-01881e6b96f7,f718e507-296b-4f22-842b-5fb91322069b,[527390],91708.0,243205.0,571201.0,Feign Death,en,2021-07-23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# add 

decklist['must_keeps'] = False

decklist.loc[decklist['type_line'].str.find('Land') != -1.0, 'must_keeps'] = True

decklist.to_csv(os.path.join('data', 'decklist.csv'), index=False)



In [16]:
def gen_sampling_ndxs(deck_ndxs:list[int], no_must_keeps:int, no_basics:int) -> list[int]:
    
    return random.sample(deck_ndxs, 99-no_must_keeps-no_basics-1)

def decklist_from_ndxs(decklist:pd.DataFrame, ndxs:list[int]) -> pd.DataFrame:
    return decklist.iloc[ndxs, :]

def add_repeats(repeats:dict, deck_ndxs:list[int]) -> list[int]:
    for key, value in repeats.items():
        deck_ndxs = sum(
            [[key] * value, deck_ndxs], []
        )
    return deck_ndxs


def sample_decklist(decklist, repeats:dict) -> list[int]:

    """Collect a random sampling of your decklist
    decklist: a pandas dataframe representing your decklist
        in a scryfall format
    repeats: a dictionary of specific decklist ndxs 
        of cards you want repeated
        ! in a future update this will be collected 
            from the number attribute in the decklist"""
    

    # drop must keeps from sampling pool
    # and generate list of ndxs

    ndxs = decklist[decklist['must_keeps'] == False].index.tolist()
    
    # randomly sample the deck based on 
    # the number of basic lands and 'must keep' cards

    ndxs = gen_sampling_ndxs(
        deck_ndxs = ndxs, 
        no_must_keeps = decklist[decklist['must_keeps'] == True].shape[0],
        no_basics = 21 # we can solve for this 
    )
    
    # add in 'must keep' cards

    ndxs = sum(
        [decklist[decklist['must_keeps'] == True].index.tolist(), ndxs] , [] 
    )

    # add in cards that have multiples

    ndxs = add_repeats(repeats, ndxs)

    return ndxs




In [17]:
# sample the following from each decklist

## initial 7 cards
## free mulligan
## mulligan to 6
## mulligan to 5

# for the previous 4 situations draw two cards from the top of the library

# just deck things

In [18]:
def draw(deck_ndxs:list[int], no_cards:int) -> list[int]:
    return deck_ndxs[0:no_cards+1]

def shuffle(deck_ndxs:list[int]) -> list[int]:
    random.shuffle(deck_ndxs) # does not return original list
    return deck_ndxs

In [19]:
repeats = {
    # decklist ndx : no_repeats
    36 : 22
}

# save index sets as a parquet from a dataframe

draw(shuffle(sample_decklist(decklist, repeats)), 7)

ValueError: Sample larger than population or is negative

# sampling format

# generating samples

In [ ]:
# constants

no_deck_samples = 1
no_samples_per_deck = 1000 # we will eventually do this to stability
no_mulligans = 3 
no_cards_in_hand = 7
cards_drawn = 2

# get sampling pool

deck = shuffle(sample_decklist(decklist, repeats))

# sample cards by
# shuffling the deck and drawing 9 cards
#   where the first 7 are the initial hand 
#   and the last two are your first two draws

samples = [
    draw(shuffle(deck), no_cards_in_hand+cards_drawn-1) 
    for i in range(no_samples_per_deck*no_mulligans)
]

# generate sampling indices

sampling_multidex = sum(sum([
    [[(m, n, i) for i in range(no_mulligans)]
    for n in range(no_samples_per_deck)]
    for m in range(no_deck_samples)
], []), [])

# generate data column names

column_names = sum(
    [
        [f'card_no_{no}' for no in range(no_cards_in_hand)],
        [f'draw_no_{no}' for no in range(cards_drawn)]
    ], []
)

data = pd.DataFrame(
    samples, 
    index=pd.MultiIndex.from_tuples(
        sampling_multidex, names=['deck_sample', 'deck_sample_no','mulligan_count']
    ),
    columns=column_names
)

# data.to_parquet(os.path.join(
#     'data', 'samples.parquet'
# ))

data

card_no_0  card_no_1  card_no_2  \
deck_sample deck_sample_no mulligan_count                                    
0           0              0                      36         36        110   
                           1                      29         37         36   
                           2                     115        119          8   
            1              0                     114         67        115   
                           1                      82        117         37   
...                                              ...        ...        ...   
            998            1                       2         86         71   
                           2                     115         36         36   
            999            0                      88         36          5   
                           1                     103         36         36   
                           2                     104         63         36   

                                           card_no_3  card_no_4  card_no_5  \
deck_sample deck_sample_no mulligan_count                                    
0           0              0                      98         40         10   
                           1                       1         80         77   
                           2                      36        109         88   
            1              0                     107        117         23   
                           1                     107         70        114   
...                                              ...        ...        ...   
            998            1                      44         36         11   
                           2                     114         29         67   
            999            0                      34        104         36   
                           1                      36         40        106   
                           2                      36         36         36   

                                           card_no_6  draw_no_0  draw_no_1  
deck_sample deck_sample_no mulligan_count                                   
0           0              0                      36        105          5  
                           1                      16        105         36  
                           2                      36         36        114  
            1              0                      36         36         86  
                           1                      29         95         55  
...                                              ...        ...        ...  
            998            1                      63         62         30  
                           2                       5         65         36  
            999            0                      21         65         36  
                           1                      86         93         36  
                           2                      23         80         34  

[3000 rows x 9 columns]

In [ ]:
pd.read_parquet(os.path.join('data','samples.parquet'))

card_no_0  card_no_1  card_no_2  \
deck_sample deck_sample_no mulligan_count                                    
0           0              0                      36         48         47   
                           1                      36         38         36   
                           2                      15         10         36   
            1              0                       4         35         80   
                           1                      21        105         67   
...                                              ...        ...        ...   
            998            1                      69         35         97   
                           2                      99         35         36   
            999            0                      51        103         97   
                           1                      60         43         13   
                           2                      96         36        106   

                                           card_no_3  card_no_4  card_no_5  \
deck_sample deck_sample_no mulligan_count                                    
0           0              0                      36         60         83   
                           1                      28         36         68   
                           2                      80         36         73   
            1              0                      33         36        113   
                           1                      36         74         81   
...                                              ...        ...        ...   
            998            1                      36         89         41   
                           2                     102         13         36   
            999            0                      36         36         36   
                           1                     113         47         32   
                           2                      36         29         33   

                                           card_no_6  draw_no_0  draw_no_1  
deck_sample deck_sample_no mulligan_count                                   
0           0              0                     105         36         59  
                           1                      37         59         29  
                           2                      61          7          4  
            1              0                      36         36        106  
                           1                     113         36        114  
...                                              ...        ...        ...  
            998            1                      36         36         80  
                           2                      43         21         36  
            999            0                      36          6         74  
                           1                       9         97         36  
                           2                      81         47         16  

[3000 rows x 9 columns]

In [ ]:
# need to add a way to map card types or whatever to the integers

def get_mappings(decklist:pd.DataFrame, column:str) -> dict:

    return decklist.loc[:,column].to_dict()


for col in data.columns:
    data[col] = data[col].map(get_mappings(decklist, 'type_line'))

data

card_no_0  \
deck_sample deck_sample_no mulligan_count                                          
0           0              0                                         Enchantment   
                           1                         Creature — Phyrexian Horror   
                           2                                   Creature — Spirit   
            1              0                                             Instant   
                           1                                             Sorcery   
...                                                                          ...   
            998            1                                             Instant   
                           2                                   Creature — Spirit   
            999            0                                                Land   
                           1                        Artifact — Equipment // Land   
                           2               Legendary Creature — Aetherborn Rogue   

                                                         card_no_1  \
deck_sample deck_sample_no mulligan_count                            
0           0              0                           Enchantment   
                           1                    Basic Land — Swamp   
                           2               Creature — Dauthi Rogue   
            1              0               Creature — Human Wizard   
                           1                  Artifact — Equipment   
...                                                            ...   
            998            1                                  Land   
                           2                           Enchantment   
            999            0                           Enchantment   
                           1                           Enchantment   
                           2                              Artifact   

                                                      card_no_2  \
deck_sample deck_sample_no mulligan_count                         
0           0              0               Artifact — Equipment   
                           1                        Enchantment   
                           2                            Instant   
            1              0                  Creature — Spirit   
                           1                 Basic Land — Swamp   
...                                                         ...   
            998            1                        Land — Gate   
                           2                        Enchantment   
            999            0                               Land   
                           1                        Enchantment   
                           2                        Enchantment   

                                                      card_no_3  \
deck_sample deck_sample_no mulligan_count                         
0           0              0               Artifact — Equipment   
                           1                            Sorcery   
                           2                        Enchantment   
            1              0                 Enchantment — Aura   
                           1                 Enchantment — Aura   
...                                                         ...   
            998            1               Artifact — Equipment   
                           2                            Instant   
            999            0                            Instant   
                           1                        Enchantment   
                           2                        Enchantment   

                                                                       card_no_4  \
deck_sample deck_sample_no mulligan_count                                          
0           0              0                            Creature — Dauthi Minion   
                           1                                   Creature

In [ ]:
# label card types

# lands


for col in data.columns:
    data.loc[data[col].str.find('Land') != -1, col] = 'land'
    data.loc[data[col].str.find('Creature') != -1, col] = 'creature'
    data.loc[data[col].str.find('Enchantment') != -1, col] = 'enchantment'
    data.loc[data[col].str.find('Equipment') != -1, col] = 'equipment' 
    data.loc[data[col].str.find('Sorcery') != -1, col] = 'sorcery'
    data.loc[data[col].str.find('Instant') != -1, col] = 'instant'
    data.loc[data[col].str.find('Artifact') != -1, col] = 'artifact'

# 

In [ ]:
# sum number of lands

categories = []

for col in data.columns:
    categories = sum(
        [[i for i in pd.unique(data[col]).tolist() 
         if i not in categories],categories],[]
    )

categories

['enchantment',
 'creature',
 'instant',
 'sorcery',
 'land',
 'artifact',
 'equipment']